## 先编译好ltp_server并在本机端口12345运行
### 参考Install_ltp.md

In [37]:
import requests as req
import json

In [54]:
# f=json/xml 代表返回格式
# t=分词（ws）,词性标注（pos），命名实体识别（ner），依存句法分析（dp），语义角色标注（srl）或者全部任务（all）
res = req.post(url='http://127.0.0.1:12345/ltp',data=
               str('f=json&t=all&s=现在苹果手机在全世界销量正在被华为手机慢慢赶超，两家公司的高管都发表了各自的看法。库克认为华为手机越来越好了。余承东说苹果手机真的贵。'
                  ).encode('utf-8'))
res_obj = json.loads(res.text)

In [60]:
res_obj[0]

[{'arg': [],
  'cont': '库克',
  'id': 0,
  'ne': 'S-Nh',
  'parent': 1,
  'pos': 'nh',
  'relate': 'SBV'},
 {'arg': [{'beg': 0, 'end': 0, 'id': 0, 'type': 'A0'},
   {'beg': 2, 'end': 6, 'id': 1, 'type': 'A1'}],
  'cont': '认为',
  'id': 1,
  'ne': 'O',
  'parent': -1,
  'pos': 'v',
  'relate': 'HED'},
 {'arg': [],
  'cont': '华为',
  'id': 2,
  'ne': 'O',
  'parent': 3,
  'pos': 'nz',
  'relate': 'ATT'},
 {'arg': [],
  'cont': '手机',
  'id': 3,
  'ne': 'O',
  'parent': 5,
  'pos': 'n',
  'relate': 'SBV'},
 {'arg': [],
  'cont': '越来越',
  'id': 4,
  'ne': 'O',
  'parent': 5,
  'pos': 'd',
  'relate': 'ADV'},
 {'arg': [{'beg': 2, 'end': 3, 'id': 0, 'type': 'A0'},
   {'beg': 4, 'end': 4, 'id': 1, 'type': 'ADV'}],
  'cont': '好',
  'id': 5,
  'ne': 'O',
  'parent': 1,
  'pos': 'a',
  'relate': 'VOB'},
 {'arg': [],
  'cont': '了',
  'id': 6,
  'ne': 'O',
  'parent': 5,
  'pos': 'u',
  'relate': 'RAD'},
 {'arg': [],
  'cont': '；',
  'id': 7,
  'ne': 'O',
  'parent': 1,
  'pos': 'wp',
  'relate': 'WP'

In [40]:
head_words = ['表示','指出','认为','坦言','告诉','看来','透露','介绍','明说','说','强调','所说','提到','说道','称','声称','建议','呼吁',
              '提及','地说','直言','普遍认为','批评','重申','提出','明确指出','觉得','宣称','猜测','特别强调','写道','引用','相信',
              '暗示','解释','谈到','深知','称赞','感慨','主张','还称','中称','指责','披露','明确提出','描述','提醒','深有体会','爆料']

In [56]:
sentence = "".join([word_obj['cont'] for word_obj in res_obj[0][0]])

In [57]:
def trim_end_period(text):
    if not text:
        return text
    _len = len(text)
    if text[_len-1] in ['。','；','！']:
        return text[:_len-1]

In [58]:
trim_end_period(sentence)

'现在苹果手机在全世界销量正在被华为手机慢慢赶超，两家公司的高管都发表了各自的看法'

In [61]:
import re

def split_sentences(text):
    sentences = re.split('(。|！|\!|\.|？|\?)',text) 
 
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        new_sents.append(sent)
    return new_sents

In [65]:
import jieba

def get_target_sentences(sentences):
    target_sentences = []
    for sent in sentences:
        words = jieba.cut(sent)
        if len([word for word in words if word in head_words]) > 0:
            target_sentences.append(sent)
    return target_sentences

In [66]:
get_target_sentences(split_sentences("现在苹果手机在全世界销量正在被华为手机慢慢赶超，两家公司的高管都发表了各自的看法。库克认为华为手机越来越好了。余承东说苹果手机真的贵。"))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Mark\AppData\Local\Temp\jieba.cache
Loading model cost 0.922 seconds.
Prefix dict has been built succesfully.


['库克认为华为手机越来越好了。', '余承东说苹果手机真的贵。']